In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
#import pandas as pd
#import numpy

### 1. Pretrained Model

In [2]:
# 사전 학습된 모델 불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"

model_org = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer_org = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# 공통 입력 문장
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [4]:
# 토큰화 테스트
print(tokenizer_org.max_model_input_sizes)

tokens_org = tokenizer_org(input_txt).tokens()
print(tokens_org)

{'gpt2': 1024, 'gpt2-medium': 1024, 'gpt2-large': 1024, 'gpt2-xl': 1024, 'distilgpt2': 1024}
['▁바람', '도', '▁없는', '▁공중에', '▁수직', '의', '▁파', '문을', '▁내', '이며', '▁고', '요', '히', '▁떨어지는', '▁오동', '잎은', '▁누', '구의', '▁발자', '취', '▁입', '니까', '.']


In [5]:
# 모델 생성 결과 테스트
max_length=128

input_ids_org = tokenizer_org(input_txt, return_tensors="pt")["input_ids"].to(device)
output_org = model_org.generate(input_ids_org, max_length=max_length, do_sample=False)
print(tokenizer_org.decode(output_org[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


In [6]:
# generate 함수 옵션별 테스트 결과 비교
output_beam1 = model_org.generate(input_ids_org, max_length=max_length, num_beams=10, no_repeat_ngram_size=2, do_sample=False)
print(tokenizer_org.decode(output_beam1[0]))

output_beam2 = model_org.generate(input_ids_org, max_length=max_length, num_beams=7, no_repeat_ngram_size=2, do_sample=True, temperature=2.0, top_k=50)
print(tokenizer_org.decode(output_beam2[0]))

output_beam3 = model_org.generate(input_ids_org, max_length=max_length, num_beams=7, no_repeat_ngram_size=2, do_sample=True, top_p=0.90)
print(tokenizer_org.decode(output_beam3[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"
바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"저것이 누구일까. 그것은 누구인가?" "모르겠지만."
"누가 저것을 보고 있는지 모르겠습니다."
"선생님도 알고 있겠지요. 그러나 저는 선생님이 아니죠. 선생은 누구인지조차 모르는 모양입니다"라고 말을 이었다.
"그렇다고 저것이 선생님을 뜻하는 것인 줄 알았어요, 그렇지 않습니까. 선생님께서는 저기 계실 때 저의 존재를 알고 계시는 겁니다. 아니요, 선생님은 어디에 계신지 아세요" 하고
바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그런데 이게 웬일입니까?"
"아니야. 그게 무슨 소리냐고."
"그래, 그건 너희들끼리 얘기하는 거 아니냐. 난 그 얘기를 들은 적이 없단다."
그렇게 말하고는,
"이런, 그런 얘기는 들어본 적이 없다."
그러자 그제야 저도 모르게 고개를 끄덕였다.
"무슨 소리야, 그거야."
그때부터 저는 한숨을 내쉬기 시작했다.
"정말


In [7]:
# 모델 저장
model_org.save_pretrained('./model/output_1_org')

### 2. SFT(Supervised Fine-Tuning)

In [8]:
import transformers
import logging
import json
import copy
from torch.utils.data import Dataset
from typing import Optional, Dict, Sequence
from dataclasses import dataclass
#from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from transformers import pipeline

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
# SFT를 위한 기본 모델 불러오기
model_sft = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer_sft = AutoTokenizer.from_pretrained(model_name,
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>', padding_side="right", model_max_length=512)

In [10]:
class SFT_dataset(Dataset):
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [11]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)

        return dict(input_ids=input_ids, labels=labels, attention_mask=input_ids.ne(self.tokenizer.pad_token_id))

In [12]:
# 데이터셋 준비
train_dataset_sft = SFT_dataset(data_path_1_SFT='/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer_sft)
data_collator_sft = DataCollatorForSupervisedDataset(tokenizer=tokenizer_sft)

In [13]:
# 학슴을 위한 파마리터 설정, 학습, 결과 저장
training_args_sft = TrainingArguments(
    output_dir="./test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )

trainer_sft = Trainer(model=model_sft, args=training_args_sft, data_collator=data_collator_sft, train_dataset=train_dataset_sft)

trainer_sft.train()
model_sft.save_pretrained('./model/output_1_sft')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.984100
1000,2.776800
1500,2.687200


In [21]:
# 결과 비교
list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

# 기본 모델
list_result_org = []
for prompt in list_prompt:
    input_ids = tokenizer_org(prompt, return_tensors="pt")["input_ids"].to(device)
    output = model_org.generate(input_ids, max_length=max_length, num_beams=4, no_repeat_ngram_size=4, do_sample=True, temperature=2.0, top_k=50)
    list_result_org.append(tokenizer_org.decode(output[0]))

# SFT 모델
generator_sft = pipeline('text-generation', model='./model/output_1_sft', tokenizer=tokenizer_sft)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

# PROMPT_DICT = {
#     "prompt_input": (
#         "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
#     )
# }

# prompt_dict = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result_sft = generator_sft(list_prompt, **generation_args)

# 결과 비교
for prompt, result_org, result_sft in zip(list_prompt, list_result_org, list_result_sft):
    print('*' * 20)
    print(f"질문(prompt): {prompt}")
    print()
    print(f"대답(기본 모델): {result_org}")
    print()
    print(f"대답(SFT  모델): {result_sft[0]['generated_text']}")
    print()

********************
질문(prompt): 불고기용 고기 한우에요?

대답(기본 모델): 불고기용 고기 한우에요?!
저거 쫄깃하면서도 매콤하며 달달한 육향이 살아있고
다이어트하는 동안에도 넘 잘어울리는 식감이였어요!
#다히먹방<unk>#먹스타그램 #존맛
#존맛탱 #불고기 #먹부림 #맛있다
#먹방 #먹짤 #f4follow #foodporn #l4l #선팔 #맞팔해요
#소통 #팔로우 #일상 #소통스타그램

대답(SFT  모델): 불고기용 고기 한우에요?\n\n저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만, 인터넷에서 검색해보시는 것을 추천드립니다!\nSure, I am an AI language model, I cannot provide more context

********************
질문(prompt): 리처드 닉슨이 43대 부통령직을 수행한 년도는?

대답(기본 모델): 리처드 닉슨이 43대 부통령직을 수행한 년도는?.
1999년 5월 22일 ~ 2004년 6월 16일.
브라이언 맥클레인 ( 1997-1997 ) : 미 공화당 대선 후보.
토니 블레어 ( 1993-2003 ) : 미국 연방 하원선거
마이크 허커 ( 1998-2004 ) : 뉴욕 주지사 ( 공화당 )
제프리 스톤 ( 2000-2003 )
찰리 폴슨 ( 2002-2011년 ) : 대만 인민위원회 위원장
제이슨 리 ( 2005-2010년 ) : 미국 뉴욕 주지사 ( 민주당 )
조셉 하딩턴 ( 2006

대답(SFT  모델): 리처드 닉슨이 43대 부통령직을 수행한 년도는?\n\n닉슨은 40대 부통령직을 수행하지 않았습니다. 그러나 그는 1960년대 후반부터 1970년대 초반까지 부통령직을 수행했습니다.子供養)使用: 英祖僧士, 英祖宗軍師, 永祖宗軍士, 榮祖宗軍事, 永

********************
질문(prompt): 시카고 오헤어 국제공항은 어디에 있어?

대답(기본 모델): 시카고 오헤어 국제공항은 어디에 있어?" 등의 반응을 보였다.

#### 분석
- 사전 학습만 이뤄진 모델과 fine-tuning된 모델의 비교에서 상대적으로 SFT 모델이 조금 더 낫다는 생각이지만, 만족스러운 응답을 보이는 것은 아님
- generation 관련해서 공통적으로 변경 가능한 파라미터가 있는데(num_beams, no_repeat_ngram_size, top_k 등), 해당 파라미터의 변경에 따른 결과 비교도 필요해 보임

## 회고
- 상업적으로도 널리 이용되고 있는 chatGPT의 동작 원리를 이해할 수 있는 시간이었음.
- Reward Model을 학습하는 단계에서 라이브러리를 불러올 때 에러가 발생하였는데, 해당 이슈를 해결하지 못해, 이후 단계를 직접 테스트해보지 못한 것이 아쉬움
- Node에서의 학습을 통해 강화학습 단계에서 정성적인 성능이 크게 향상되는 것을 확인하였음.